In [ ]:
!pip install google-cloud
!pip install google-cloud-videointelligence
from google.colab import drive

drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
from google.cloud import videointelligence_v1 as videointelligence
import os
import io
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/drive/MyDrive/Google-cloud-video-intelligence-api-file/video-intelligence-325504-fc7cd33ccbf8.json"

In [ ]:
#Detect person including attributes from every segment of video file
def detect_person(local_file_path):
    client = videointelligence.VideoIntelligenceServiceClient()

    with io.open(local_file_path, "rb") as f:
        input_content = f.read()

    # Configure the request
    config = videointelligence.types.PersonDetectionConfig(
        include_bounding_boxes=True,
        include_attributes=True,
        include_pose_landmarks=True,
    )
    context = videointelligence.types.VideoContext(person_detection_config=config)

    # Start the asynchronous request
    operation = client.annotate_video(
        request={
            "features": [videointelligence.Feature.PERSON_DETECTION],
            "input_content": input_content,
            "video_context": context,
        }
    )

    print("\nProcessing video for person detection annotations.")
    result = operation.result(timeout=300)
    return result

In [ ]:
#Person is detected by recognizing positions of various body parts
#Relative position of every body part can be used as metadata of that segment 
def get_relative_measures(response1):
  motion_detection_data = []
  for annotation in response1.annotation_results[0].person_detection_annotations:
    for track in annotation.tracks:
      for timestamped_obj in track.timestamped_objects:
        land_marks_coordinates = []
        for land_mark in timestamped_obj.landmarks:
          land_marks_coordinates.append([land_mark.point.x,land_mark.point.y])
        relative_measures = []
        for i in range(len(land_marks_coordinates)):
          for j in range(i+1,len(land_marks_coordinates)):
            relative_measures.append([land_marks_coordinates[i][0]-land_marks_coordinates[j][0],land_marks_coordinates[i][1]-land_marks_coordinates[j][1]])
        motion_detection_data.append(relative_measures)
  return motion_detection_data

In [ ]:
#Manhattan distance between relative position of same body part in repsective segment can be used as metric for determining body posture similarity
def motion_difference_score(motion_data1,motion_data2):
  result = 0
  count = 0
  for segment1 in motion_data1:
    similarity_score = 1e9
    for segment2 in motion_data2:
      if len(segment1)!=len(segment2):
        continue 
      score = 0
      for i in range(len(segment1)):
        score+=(abs(segment1[i][0]-segment2[i][0]) + abs(segment1[i][1]-segment2[i][1]))/2 
      similarity_score = min(similarity_score,score)
    count+=1
    result+=similarity_score 
  return result/count 

In [ ]:
realtive_measures_data = []

root_dir = "/content/drive/MyDrive/dance-videos/"
for video_path in os.listdir(root_dir):
  response = detect_person(root_dir + video_path)
  relative_measures = get_relative_measures(response)
  realtive_measures_data.append([os.path.basename(video_path),relative_measures])


Processing video for person detection annotations.

Processing video for person detection annotations.

Processing video for person detection annotations.

Processing video for person detection annotations.

Processing video for person detection annotations.


In [ ]:
threshold = 10.0

for name1,msr1 in realtive_measures_data:
  for name2,msr2 in realtive_measures_data:
    score = motion_difference_score(msr1,msr2)
    if score<=threshold:
      print(f"{name1}  VS  {name2}  ==>  {score}")

gen_results_1.mp4  VS  gen_results_1.mp4  ==>  0.0
gen_results_1.mp4  VS  gen_results_3.mp4  ==>  4.737978315940409
gen_results_1.mp4  VS  gen_results_2.mp4  ==>  4.025387151900566
gen_results_1.mp4  VS  gen_results_4.mp4  ==>  3.7278370367306652
gen_results_1.mp4  VS  gen_results_5.mp4  ==>  3.1645116422212487
gen_results_3.mp4  VS  gen_results_1.mp4  ==>  4.679918041383779
gen_results_3.mp4  VS  gen_results_3.mp4  ==>  0.0
gen_results_3.mp4  VS  gen_results_2.mp4  ==>  3.6118425877005964
gen_results_3.mp4  VS  gen_results_4.mp4  ==>  3.0655248048128905
gen_results_3.mp4  VS  gen_results_5.mp4  ==>  3.718429116959925
gen_results_2.mp4  VS  gen_results_2.mp4  ==>  0.0
gen_results_2.mp4  VS  gen_results_4.mp4  ==>  3.2787154484234873
gen_results_2.mp4  VS  gen_results_5.mp4  ==>  3.7005139321821634
gen_results_4.mp4  VS  gen_results_2.mp4  ==>  3.232380141343536
gen_results_4.mp4  VS  gen_results_4.mp4  ==>  0.0
gen_results_4.mp4  VS  gen_results_5.mp4  ==>  3.5909635134298226
gen_resul

In [ ]:
"More to Come"